# Extracting Structured Data from Images
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/togethercomputer/together-cookbook/blob/main/Structured_Text_Extraction_from_Images.ipynb)

## Introduction

In this notebook we will demonstrate how you can use a language vision model(Llama 3.2 90B Vision) along with an LLM that has JSON mode enabled(Llama 3.1 70B) to extract structured text from images.

In our case we will extract line items from an invoice in the form of a JSON.

<img src="https://github.com/togethercomputer/together-cookbook/blob/main/images/structured_text_image.png?raw=1" width="750">


### Install relevant libraries

In [1]:
!pip install together

In [2]:
import together, os

# Paste in your Together AI API Key or load it
TOGETHER_API_KEY = os.environ.get("0d2269b42fce3a1a9f64bc28898267fb469e18429614ced15f4fd082657d0a4f")

## Create Invoice Structure using Pydantic

We need a way of telling the LLM what structure to organize information into - including what information to expect in the receipt. We will do this using `pydantic` models.

Below we define the required classes.

- Each line item on the receipt will have a `name`, `price` and `quantity`. The `Item` class specifies this.
- Each receipt/invoice is a combination of multiple line `Item` elements along with a `total` price. The `Receipt` class specifies this.

In [3]:
import json
from pydantic import BaseModel, Field

class Item(BaseModel):
    name: str
    price: float
    quantity: int = Field(default=1)

class Receipt(BaseModel):
    items: list[Item]
    total: float

## Lets bring in the reciept that we want to extract information from

Notice that this is a real receipt with multiple portions that are not relevant to the line item extraction structure we've outlined above.

<img src="https://ocr.space/Content/Images/receipt-ocr-original.webp" height="500">

## 1. Extract Information Receipt

We will use the Llama 3.2 90B Vision model to extract out information in normal text format.

In [7]:
from together import Together

getDescriptionPrompt = "Extract out the details from each line item on the receipt image. Identify the name, price and quantity of each item. Also specify the total."

imageUrl = "https://daytodayemail.com/wp-content/uploads/2022/08/doctor-prescription-template-78896506-768x756.png"

client = Together(api_key="0d2269b42fce3a1a9f64bc28898267fb469e18429614ced15f4fd082657d0a4f")

response = client.chat.completions.create(
    model="meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": getDescriptionPrompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": imageUrl,
                    },
                },
            ],
        }
    ],
)

info = response.choices[0].message.content

In [8]:
print(info)

The image does not display a receipt with line items, prices, and quantities. Instead, it appears to be a template for an Ayurvedic doctor's prescription standard format. The template includes sections for the doctor's information, patient details, prescription specifics, refill instructions, and authentication. There are no line items, prices, or quantities listed on this document.

**No Line Items Found**

*   No line items are present in the image.
*   No prices or quantities are provided for any items.

**Total Not Applicable**

*   Since there are no line items, a total is not applicable in this context.

In summary, the image does not contain a receipt with line items, prices, and quantities, but rather a template for an Ayurvedic doctor's prescription standard format. Therefore, the requested information cannot be extracted from this image.


Notice that the model is not perfect and wasn't able to extract out some line items. It's hard for most models to perform this zero-shot extraction of data from images. A way to improve this is to finetune the model using [Visual Intruction Tuning](https://arxiv.org/abs/2304.08485).

## 2. Organize Information as JSON

We will use Llama 3.1 70B with structured generation in JSON mode to organize the information extracted by the vision model into an acceptable JSON format that can be parsed.

`Meta-Llama-3.1-70B-Instruct-Turbo` will strcitly respect the JSON schema passed to it.

In [ ]:
extract = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "The following is a detailed description of all the items, prices and quantities on a receipt. Extract out information. Only answer in JSON.",
            },
            {
                "role": "user",
                "content": info,
            },
        ],
        model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
        response_format={
            "type": "json_object",
            "schema": Receipt.model_json_schema(),
        },
    )

In [ ]:
output = json.loads(extract.choices[0].message.content)
print(json.dumps(output, indent=2))

Althought with some missed line items we were able to extract out structured JSON from an image in a zero shot manner! To improve the results for your pipeline and make them production ready I recommend you [finetune](https://docs.together.ai/docs/fine-tuning-overview) the vision model on your own dataset!

Learn more about how to use JSON mode in the [docs](https://docs.together.ai/docs/json-mode) here!